In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/pbcquoc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_data(path):
    df = pd.read_csv(path, header=None)
    X = df[0].values
    y = df[1].values
    x_tok = Tokenizer(char_level=True, filters='')
    x_tok.fit_on_texts(X)
    y_tok = Tokenizer(char_level=True, filters='')
    y_tok.fit_on_texts(y)
    
    X = x_tok.texts_to_sequences(X)
    y = y_tok.texts_to_sequences(y)
    
    X = pad_sequences(X)
    y = np.asarray(y)
    
    return X, y, x_tok.word_index, y_tok.word_index

X, y, x_wid, y_wid= load_data('data/data.csv')

In [3]:
hidden_size = 128
input_size = len(x_wid)
output_size = len(y_wid)
max_length = y.shape[1]
print("input vocab: {} - output vocab: {} - length of target: {}".format(input_size, output_size, max_length))

input vocab: 34 - output vocab: 11 - length of target: 10


In [4]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size + 1, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    
    def forward(self, input):
        # input: SxB        
        embedded = self.embedding(input)
        output, hidden = self.gru(embedded)
        return output, hidden # SxBxH, 1xBxH              

class Attn(nn.Module):
    def __init__(self, hidden_size):
        super(Attn ,self).__init__()
        
    def forward(self, hidden, encoder_outputs):
        # encoder_outputs: TxBxH
        # hidden: SxBxH
        encoder_outputs = torch.transpose(encoder_outputs, 0, 1) #BxTxH
        hidden = torch.transpose(torch.transpose(hidden, 0, 1), 1, 2) # BxHxS
        energies = torch.bmm(encoder_outputs, hidden) # BxTxS
        energies = torch.transpose(energies, 1, 2) # BxSxT
        attn_weights = F.softmax(energies, dim=-1) #BxSxT
        
        output = torch.bmm(attn_weights, encoder_outputs) # BxSxH
        output = torch.transpose(output, 0, 1) # SxBxH
        attn_weights = torch.transpose(attn_weights, 0, 1) #SxBxT
        
        return output, attn_weights
    
class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size, dropout):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.embedding = nn.Embedding(output_size + 1, hidden_size)
        self.attn = Attn(hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.concat = nn.Linear(self.hidden_size*2, hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input, hidden, encoder_outputs):
        # input: SxB
        # encoder_outputs: BxSxH
        # hidden: 1xBxH
        embedded = self.embedding(input) # 1xBxH
        embedded = self.dropout(embedded)
        rnn_output, hidden = self.gru(embedded, hidden)  #SxBxH, 1xBxH
        context, attn_weights = self.attn(rnn_output, encoder_outputs) # SxBxH
        concat_input = torch.cat((rnn_output, context), -1)
        concat_output = torch.tanh(self.concat(concat_input)) #SxBxH
        
        output = self.out(concat_output) # SxBxoutput_size
        return output, hidden, attn_weights


In [15]:
encoder = Encoder(input_size, hidden_size)
decoder = Decoder(output_size, hidden_size, 0.1)
input = torch.randint(1, input_size, (34, 10))
encoder_outputs, hidden = encoder(input)
input_decoder = torch.randint(1, output_size, (7, 10))
output, hidden, attn_weights = decoder(input_decoder, hidden, encoder_outputs)
print(output.size())

torch.Size([7, 10, 11])


In [8]:
def train(inputs, targets, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 # Added onto for each word
    
    decoder_inputs = targets[:-1]
    decoder_targets = targets[1:]
    
    encoder_outputs, encoder_hidden = encoder(inputs)
    output, hidden, attn_weights = decoder(decoder_inputs, encoder_hidden, encoder_outputs)
    
    loss = nn.

    